## Hospital General Information Dataset

### By: Anurag Bolneni

This notebook has our preliminary hospital reccomendation system MVP based on the Hospital General Information dataset from CMS. It is split into two sections:
+ Step 1: Data Cleaning & Manipulation
+ Step 2: Taking User Input
+ Step 3: Reccomendation System MVP

## Step 1: Data Cleaning & Manipulation

We first import necessary libraries and use an API to pull data from CMS websites. Then, data is fed into a pandas dataframe where we clean for parameters of interest and data types of different columns.

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
pd.read_csv('Data/Hospital_General_Information.csv').head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,1,9,1,NaN,8,8,NaN,14,11,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,9,1,NaN,8,8,NaN,14,14,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,Acute Care Hospitals,Proprietary,...,1,7,1,NaN,8,8,NaN,14,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,6,0,NaN,8,8,NaN,14,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,4,0,NaN,8,Not Available,5.0,14,8,NaN


In [3]:
df_hosp_gen_info = pd.read_csv('Data/Hospital_General_Information.csv').iloc[:,:13].drop(columns=['Phone Number','Meets criteria for promoting interoperability of EHRs'])
df_hosp_gen_info.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,Acute Care Hospitals,Proprietary,Yes,2
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes,2


In [4]:
print('Length of unique Facility ID vs total:',len(df_hosp_gen_info['Facility ID'].unique()),',',len(df_hosp_gen_info['Facility ID']))
print('Length of unique address vs total:',len(df_hosp_gen_info.Address.unique()),',',len(df_hosp_gen_info.Address))
print('Variations in hospital overall ratings:', Counter(df_hosp_gen_info['Hospital overall rating']))
print('Emerg_services:', Counter(df_hosp_gen_info['Emergency Services']))
df_hosp_gen_info.dtypes

Length of unique Facility ID vs total: 5306 , 5306
Length of unique address vs total: 5276 , 5306
Variations in hospital overall ratings: Counter({'Not Available': 1996, '3': 1006, '4': 979, '2': 682, '5': 452, '1': 191})
Emerg_services: Counter({'Yes': 4455, 'No': 851})


Facility ID                object
Facility Name              object
Address                    object
City                       object
State                      object
ZIP Code                    int64
County Name                object
Hospital Type              object
Hospital Ownership         object
Emergency Services         object
Hospital overall rating    object
dtype: object

-- Anurag to write blurb about above findings later

We'll be taking the cosine similarity of State and Emergency Services to test our MVP of the reccomendation algorithm. As a last step of cleaning, we need to change Emergency Services (Y/N) to binary classifications.

In [5]:
df_hosp_gen_info['Emergency Services'] = [1 if x=='Yes' else 0 for x in df_hosp_gen_info['Emergency Services']]

In [6]:
df_HCAHPS = pd.read_csv('Data/HCAHPS-Hospital.csv')
df_HCAHPS = df_HCAHPS[['Facility ID','HCAHPS Measure ID','HCAHPS Question','HCAHPS Answer Percent']]
df_HCAHPS['HCAHPS Answer Percent'] = pd.to_numeric(df_HCAHPS['HCAHPS Answer Percent'], errors='coerce')
df_HCAHPS = df_HCAHPS.dropna(axis=0)

df_info = df_HCAHPS.groupby(['Facility ID']).count()
df = df_info[df_info['HCAHPS Question'] == 72].reset_index()
VALID_FACILITY_IDS = list(df['Facility ID'])

df_HCAHPS = df_HCAHPS[df_HCAHPS['Facility ID'].isin(VALID_FACILITY_IDS) == True]

TEST_ = df_HCAHPS[df_HCAHPS['Facility ID'].isin(VALID_FACILITY_IDS) == True]

In [7]:
%%time
# Here's how to calculate summary metrics using group_by and creating some new columns
question_type_dict = {'H_COMP_1_A_P': "nurses", 
                      'H_NURSE_RESPECT_A_P': "nurses", 
                      'H_NURSE_LISTEN_A_P': "nurses", 
                      'H_NURSE_EXPLAIN_A_P': "nurses",
                      'H_COMP_2_A_P': "doctors",
                      'H_DOCTOR_RESPECT_A_P': "doctors",
                      'H_DOCTOR_LISTEN_A_P': "doctors", 
                      'H_DOCTOR_EXPLAIN_A_P': "doctors",
                      'H_COMP_3_A_P': "patients", 
                      'H_CALL_BUTTON_A_P': "patients", 
                      'H_BATH_HELP_A_P': "patients", 
                      'H_COMP_5_A_P': "staffs", 
                      'H_MED_FOR_A_P': "staffs", 
                      'H_SIDE_EFFECTS_A_P': "staffs"
                     }
df_HCAHPS["measurement_type"] = df_HCAHPS.apply(lambda row: question_type_dict[row["HCAHPS Measure ID"]] if row["HCAHPS Measure ID"] in question_type_dict.keys() else "UNKNOWN", axis=1)
grouped = df_HCAHPS.groupby(['Facility ID', 'measurement_type']).mean()
grouped = grouped.drop("UNKNOWN", level="measurement_type").reset_index()

CPU times: user 2.17 s, sys: 35.6 ms, total: 2.21 s
Wall time: 2.21 s


In [8]:
df_HCAHPS

,Facility ID,HCAHPS Measure ID,HCAHPS Question,HCAHPS Answer Percent,measurement_type
0,010001,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...",77.0,nurses
1,010001,H_COMP_1_SN_P,"Patients who reported that their nurses ""Somet...",7.0,UNKNOWN
2,010001,H_COMP_1_U_P,"Patients who reported that their nurses ""Usual...",16.0,UNKNOWN
5,010001,H_NURSE_RESPECT_A_P,"Patients who reported that their nurses ""Alway...",81.0,nurses
6,010001,H_NURSE_RESPECT_SN_P,"Patients who reported that their nurses ""Somet...",7.0,UNKNOWN
...,...,...,...,...,...
302612,360131,H_HSP_RATING_7_8,Patients who gave their hospital a rating of 7...,20.0,UNKNOWN
302613,360131,H_HSP_RATING_9_10,Patients who gave their hospital a rating of 9...,69.0,UNKNOWN
302616,360131,H_RECMND_DN,"Patients who reported NO, they would probably ...",9.0,UNKNOWN
302617,360131,H_RECMND_DY,"Patients who reported YES, they would definite...",67.0,UNKNOWN


In [9]:
d = ['nurses', 'doctors', 'staffs', 'patients']
clinician_df = grouped[['Facility ID']]

for i in range(4):
    CLINICIANS = grouped[grouped['measurement_type']==d[i]][['Facility ID', 'HCAHPS Answer Percent']]
    CLINICIANS.rename(columns = {'HCAHPS Answer Percent':d[i]}, inplace = True)
    CLINICIANS[d[i]] = CLINICIANS[d[i]]
    clinician_df = clinician_df.merge(CLINICIANS, on='Facility ID')
    
clinician_df = clinician_df.drop_duplicates().dropna()


In [10]:
new = df_hosp_gen_info.merge(clinician_df, on='Facility ID').dropna().reset_index()
print('We have '+ str(len(new)) + ' facilties in the df after joining and dropping NA')
new = new.drop(labels=['index','Hospital Type','Hospital Ownership'], axis=1)
new

We have 2410 facilties in the df after joining and dropping NA


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Emergency Services,Hospital overall rating,nurses,doctors,staffs,patients
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,1,3,77.00,80.75,63.666667,60.666667
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,1,2,75.00,84.00,56.000000,47.666667
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,1,2,69.00,77.00,55.333333,44.666667
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,1,2,84.00,85.75,77.333333,72.666667
4,010011,ST. VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,1,2,77.25,79.25,63.000000,65.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,360118,OHIOHEALTH MANSFIELD HOSPITAL,335 GLESSNER AVENUE,MANSFIELD,OH,44903,RICHLAND,1,4,79.00,78.75,59.000000,63.666667
2406,360121,COMMUNITY HOSPITALS AND WELLNESS CENTERS,433 WEST HIGH STREET,BRYAN,OH,43506,WILLIAMS,1,3,85.75,80.50,68.000000,80.666667
2407,360123,UH ST JOHN MEDICAL CENTER,29000 CENTER RIDGE ROAD,WESTLAKE,OH,44145,CUYAHOGA,1,4,82.25,80.25,66.000000,71.333333
2408,360125,ASHTABULA COUNTY MEDICAL CENTER,2420 LAKE AVENUE,ASHTABULA,OH,44004,ASHTABULA,1,4,80.75,75.75,56.000000,70.333333


## Step 2: Take User Input

In [11]:
columns = ['Hospital overall rating','Emergency Services','State']
df = pd.DataFrame(columns = columns,dtype=object)

def user_input(df):
    # Take user input for a series of factors
    return df.append({
        'Hospital overall rating': int(input('Please rate your desired hospital on scale of 1-5:   ')),
        'Emergency Services': int(input('Do you need emergency services? (Yes = 1, No = 0)     ')),
        'State':str(input('Which State do you live in?     ')),
        'Doctors': int(input('Rate your ideal doctor on a scale of 0-100%')),
        'Nurses': int(input('Rate your ideal nurses on a scale of 0-100%')),
        'Patients': int(input('Rate your ideal patients on a scale of 0-100%')),
        'Staffs': int(input('Rate your ideal staffs on a scale of 0-100%'))},
    ignore_index = True)

In [12]:
user_input_df = user_input(df)
user_input_df.head()

Please rate your desired hospital on scale of 1-5:   4
Do you need emergency services? (Yes = 1, No = 0)     1
Which State do you live in?     MI
Rate your ideal doctor on a scale of 0-100%86
Rate your ideal nurses on a scale of 0-100%54
Rate your ideal patients on a scale of 0-100%39
Rate your ideal staffs on a scale of 0-100%06


,Hospital overall rating,Emergency Services,State,Doctors,Nurses,Patients,Staffs
0,4,1,MI,86.0,54.0,39.0,6.0


##### Step 3: Reccommendar System MVP

Our hospital reccomendation system utilizes the cleaned dataset from Step 1 and the User Input from Step 2. We use cosine similarity to determine the top hospitals based on the users target inputs. This section is broken into a set of function that to conduct vectorization of string parmaters as well as numeric parameters to determine cosine similarity of hospitals relative to a user's need. 

For our MVP, we tested our recommender sytem on Emergency Services (Y/N) and State of hospital as initial inputs. Finally, we sort the outcomes by cosine to determine the list of best hospitals. The outcomes so far look like it works relatively well, but we'll need to see how this fluctuates with additional parameters and available data.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def give_me_hospitals(df_hospital, df_patient):
    
    #Collaborative Filtering
    Patient_Preferences = ['State','Emergency Services']
    for values in Patient_Preferences:
        df_hospital = df_hospital[df_hospital[values] == df_patient[values][0]]

    df_hospital = df_hospital[df_hospital['Hospital overall rating'] == str(df_patient['Hospital overall rating'][0])]
    
    # Begin attaining X parameters directly based on hospital DF
    X_no2 = np.array([df_hospital['doctors'],
                      df_hospital['nurses'],
                      df_hospital['patients'],
                      df_hospital['staffs']]).reshape([-1,4], order='F')
    
    # Begin attaining Y paramters directly based on patient DF
    Y_Final = np.array([df_patient['Doctors'],
                        df_patient['Nurses'],
                        df_patient['Patients'],
                        df_patient['Staffs']]).reshape(-1,4)

    Cos = cosine_similarity(X_no2,Y_Final)
    df_hospital['Cosine Similarity'] = [values[0] for values in Cos]

    return df_hospital.sort_values('Cosine Similarity', ascending=False).reset_index()

In [15]:
df_hospital = new
df_patient = user_input_df
give_me_hospitals(df_hospital, df_patient)

,index,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Emergency Services,Hospital overall rating,nurses,doctors,staffs,patients,Cosine Similarity
0,1569,230029,ST JOSEPH MERCY OAKLAND,44405 WOODWARD AVE,PONTIAC,MI,48341,OAKLAND,1,4,75.25,73.00,56.000000,54.000000,0.878922
1,1619,230176,BEAUMONT HOSPITAL - TRENTON,5450 FORT STREET,TRENTON,MI,48183,WAYNE,1,4,79.75,75.25,58.000000,63.000000,0.877675
2,1566,230021,"LAKELAND HOSPITAL, ST JOSEPH",1234 NAPIER AVENUE,ST JOSEPH,MI,49085,BERRIEN,1,4,79.75,80.00,58.333333,63.333333,0.873824
3,1563,230017,BRONSON METHODIST HOSPITAL,601 JOHN STREET,KALAMAZOO,MI,49007,KALAMAZOO,1,4,83.75,83.25,67.000000,71.000000,0.860447
4,1564,230019,"ASCENSION PROVIDENCE HOSPITAL, SOUTHFIELD AND ...",16001 W NINE MILE RD,SOUTHFIELD,MI,48075,OAKLAND,1,4,76.75,75.25,58.000000,63.333333,0.860359
5,1631,230236,METRO HEALTH HOSPITAL,"5900 BYRON CENTER AVENUE, SW",WYOMING,MI,49519,KENT,1,4,78.50,77.75,63.000000,67.333333,0.858244
6,1621,230193,MCLAREN LAPEER REGION,1375 N MAIN ST,LAPEER,MI,48446,LAPEER,1,4,76.00,69.25,57.000000,55.000000,0.854622
7,1628,230222,MIDMICHIGAN MEDICAL CENTER-MIDLAND,4000 WELLNESS DRIVE,MIDLAND,MI,48670,MIDLAND,1,4,80.00,77.25,64.333333,70.333333,0.854409
8,1659,231327,SPARROW EATON HOSPITAL,321 E HARRIS STREET,CHARLOTTE,MI,48813,EATON,1,4,81.75,74.00,44.333333,64.666667,0.852120
9,1583,230059,MERCY HEALTH SAINT MARY'S,200 JEFFERSON AVENUE SE,GRAND RAPIDS,MI,49503,KENT,1,4,82.75,80.00,63.333333,65.000000,0.848629
